In [1]:
import numpy as np # used for arrays

import gym # pull the environment

import time # to get the time

import math # needed for calculations

In [2]:
env = gym.make("CartPole-v1")
print(env.action_space.n)

2


In [3]:
'''
Observation:
        Type: Box(4)
        Num     Observation               Min                     Max
        0       Cart Position             -4.8                    4.8
        1       Cart Velocity             -Inf                    Inf
        2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)
        3       Pole Angular Velocity     -Inf                    Inf
    Actions:
        Type: Discrete(2)
        Num   Action
        0     Push cart to the left
        1     Push cart to the right
        Note: The amount the velocity that is reduced or increased is not
        fixed; it depends on the angle the pole is pointing. This is because
        the center of gravity of the pole increases the amount of energy needed
        to move the cart underneath it
'''

# Learning Rate: learning rate is associated with how big you take a leap
lr = 0.1

#Discount Factor
gamma = 0.95

#Amount of iterations we are going to run until we see our model is trained
epochs = 60000
total_time = 0
total_reward = 0
prev_reward = 0

Observation = [30, 30, 50, 50]
step_size = np.array([0.25, 0.25, 0.01, 0.1])

# epsilon is associated with how random you take an action.
epsilon = 1

#exploration is decaying and we will get to a state of full exploitation
epsilon_decay_value = 0.99995

In [4]:
#randomly initializing values in our q table our q table
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape
print(q_table[0][0])

[[[0.74722875 0.28324679]
  [0.9398173  0.76022376]
  [0.7112307  0.35212502]
  ...
  [0.02773251 0.1938166 ]
  [0.74045475 0.99613517]
  [0.46113503 0.98897351]]

 [[0.44710506 0.72808767]
  [0.18018294 0.53075297]
  [0.49564318 0.43850144]
  ...
  [0.53332124 0.98937067]
  [0.34462578 0.6719122 ]
  [0.25566221 0.46102021]]

 [[0.40732536 0.07767555]
  [0.76168508 0.08206577]
  [0.60961839 0.48799963]
  ...
  [0.51073226 0.00168121]
  [0.45780774 0.46315303]
  [0.61326796 0.31180428]]

 ...

 [[0.70617826 0.20683258]
  [0.65451101 0.50292549]
  [0.52050943 0.54322778]
  ...
  [0.03860294 0.64955119]
  [0.79204953 0.24459593]
  [0.262935   0.91095703]]

 [[0.81165652 0.09784694]
  [0.2474812  0.94389301]
  [0.78449587 0.66316326]
  ...
  [0.34294385 0.23249348]
  [0.19998348 0.81215252]
  [0.2534052  0.45525725]]

 [[0.74183806 0.6895331 ]
  [0.35723788 0.5072197 ]
  [0.28974844 0.70503198]
  ...
  [0.60410199 0.28402171]
  [0.8938225  0.28108372]
  [0.53567595 0.78731454]]]


In [5]:
def get_discrete_state(state):
    discrete_state = state/step_size+ np.array([15,10,1,10])
    return tuple(discrete_state.astype(np.int))

In [6]:
#iterate through our epochs
for epoch in range(epochs + 1): 
    #set the initial time, so we can calculate how much each action takes
    t_initial = time.time() 
    
    #get the discrete state for the restarted environment, so we know what's going on
    discrete_state = get_discrete_state(env.reset()) 
    
    #we create a boolean that will tell us whether our game is running or not
    done = False
    
    #our reward is intialized at zero at the beginning of every eisode
    epoch_reward = 0 

    #Every 1000 epochs we have an episode
    if epoch % 1000 == 0: 
        print("Episode: " + str(epoch))

    while not done: 
        #Now we are in our gameloop
        #if some random number is greater than epsilon, then we take the best possible action we have explored so far
        if np.random.random() > epsilon:

            action = np.argmax(q_table[discrete_state])
        
        #else, we will explore and take a random action
        else:

            action = np.random.randint(0, env.action_space.n) 

        #now we will intialize our new_state, reward, and done variables
        new_state, reward, done, _ = env.step(action) 
    
        epoch_reward += reward 
        
        #we discretize our new state
        new_discrete_state = get_discrete_state(new_state)
        
        #we render our environment after 2000 steps
        if epoch % 2000 == 0: 
            env.render()

        #if the game loop is still running update the q-table
        if not done:
            max_new_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - lr) * current_q + lr * (reward + gamma* max_new_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state
    # if our epsilon is greater than .05m , and if our reward is greater than the previous and if we reached past our 10000 epoch, we recalculate episilon
    
    if epsilon > 0.05: 
        if epoch_reward > prev_reward and epoch > 10000:
            epsilon = math.pow(epsilon_decay_value, epoch - 10000)

            if epoch % 500 == 0:
                print("Epsilon: " + str(epsilon))

    #we calculate the final time
    tfinal = time.time() 
    
    #total epoch time
    episode_total = tfinal - t_initial 
    total_time += episode_total
    
    #calculate and update rewards
    total_reward += epoch_reward
    prev_reward = epoch_reward

    #every 1000 episodes print the average time and the average reward
    if epoch % 1000 == 0: 
        mean = total_time / 1000
        print("Time Average: " + str(mean))
        total_time = 0

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

env.close()

Episode: 0


/tmp/ipykernel_56543/1130559033.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return tuple(discrete_state.astype(np.int))


Time Average: 0.0008616750240325928
Mean Reward: 0.012
Episode: 1000
Time Average: 0.000532735824584961
Mean Reward: 21.414
Episode: 2000
Time Average: 0.001082008123397827
Mean Reward: 21.761
Episode: 3000
Time Average: 0.0005687038898468018
Mean Reward: 21.868
Episode: 4000
Time Average: 0.0009302968978881836
Mean Reward: 22.332
Episode: 5000
Time Average: 0.000547583818435669
Mean Reward: 22.638
Episode: 6000
Time Average: 0.0011672506332397461
Mean Reward: 22.097
Episode: 7000
Time Average: 0.0005508220195770264
Mean Reward: 22.747
Episode: 8000
Time Average: 0.0008356878757476806
Mean Reward: 22.38
Episode: 9000
Time Average: 0.0005490899085998536
Mean Reward: 22.145
Episode: 10000
Time Average: 0.0006819193363189698
Mean Reward: 22.907
Episode: 11000
Time Average: 0.000538360595703125
Mean Reward: 21.921
Episode: 12000
Time Average: 0.0008269457817077637
Mean Reward: 23.412
Epsilon: 0.8824941446941661
Episode: 13000
Epsilon: 0.8607047486686201
Time Average: 0.0006817357540130616
